In [1]:
from os import listdir
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import torch
from torch.utils.data import DataLoader

import Sources.DataNasaFan as DataTool
from Sources.ModelsNasaFan import RNNModel, GRUModel, LSTMModel

import AdversarialRulLib.Utils as UtilsTool
import AdversarialRulLib.Attacks as AttacksTool
import AdversarialRulLib.Defense as DefenseTool

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

TrainMode = 0

DatasetPath = 'DatasetNasaFan/'
ModelPath = 'Models/'
Window = 80

In [3]:
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {Device}")

Using device: cuda


In [4]:
listdir(DatasetPath)

DfTrain = pd.read_csv(DatasetPath + 'train_FD001.txt', header = None, sep = ' ')
DfTest = pd.read_csv(DatasetPath + 'test_FD001.txt', header = None, sep = ' ')
RulTest = pd.read_csv(DatasetPath + 'RUL_FD001.txt', header = None)

In [5]:
DfTrain, DfTest, RulTest = DataTool.SetDatasets(DfTrain, DfTest, RulTest)

In [6]:
MinMaxDict = DataTool.GetMinMaxDictionnary(DfTrain)
NdfTrain  = DataTool.NormalizeDataset(DfTrain, MinMaxDict)    
NdfTest  = DataTool.NormalizeDataset(DfTest, MinMaxDict)   

In [7]:
NbFeatures = len([c for c in NdfTrain.columns if 's' in c])

In [8]:
np.random.seed(5)
torch.manual_seed(5)

Units = np.arange(1,101)
TrainUnits = list(np.random.choice(Units, 80, replace = False))
ValUnits = list(set(Units) - set(TrainUnits))

TrainData = NdfTrain[NdfTrain['unit'].isin(TrainUnits)].copy()
ValData = NdfTrain[NdfTrain['unit'].isin(ValUnits)].copy()

TrainIndices = list(TrainData[(TrainData['rul'] >= (Window - 1)) & (TrainData['time'] > 10)].index)
ValIndices = list(ValData[(ValData['rul'] >= (Window - 1)) & (ValData['time'] > 10)].index)

TrainDataset = DataTool.data(TrainIndices, NdfTrain, Window)
ValidationDataset = DataTool.data(ValIndices, NdfTrain, Window)

UnitBySize = NdfTest.groupby('unit')['time'].max().tolist()
Units = []
for i in range(len(UnitBySize)):
    if UnitBySize[i] > Window:
        Units.append(i+1)

TestDataset = DataTool.test(NdfTest, Window)

In [9]:
print("Train", len(TrainDataset))
print("Val", len(ValidationDataset))
print("Test", len(TestDataset))

Train 9615
Val 2116
Test 77


In [10]:
TrainLoader = DataLoader(TrainDataset, batch_size = 64, shuffle = True)
ValidationLoader = DataLoader(ValidationDataset, batch_size = len(ValIndices), shuffle = True)
TestLoader = DataLoader(TestDataset, batch_size = 100)

In [11]:
if TrainMode == 1:
    epochs = 40
    learning_rate= 0.01
    
    MyRNNModel = RNNModel().to(Device)
    UtilsTool.InitModel(MyRNNModel, Device)
    UtilsTool.TrainModel(MyRNNModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)        
    torch.save(MyRNNModel, ModelPath + 'MyRNNmodel.pth')

    MyGRUModel = GRUModel().to(Device)
    UtilsTool.InitModel(MyGRUModel, Device)
    UtilsTool.TrainModel(MyGRUModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)        
    torch.save(MyGRUModel, ModelPath + 'MyGRUmodel.pth')

    MyLSTMModel = LSTMModel().to(Device)
    UtilsTool.InitModel(MyLSTMModel, Device)
    UtilsTool.TrainModel(MyLSTMModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)     
    torch.save(MyLSTMModel, ModelPath + 'MyLSTMmodel.pth')
    
else:     
    MyRNNModel = torch.load(ModelPath + 'MyRNNmodel.pth').to(Device)
    MyGRUModel = torch.load(ModelPath + 'MyGRUmodel.pth').to(Device)
    MyLSTMModel = torch.load(ModelPath + 'MyLSTMmodel.pth').to(Device)


![](./animation3.gif)

In [12]:
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyRNNModel, TestLoader, Device)
print(f'RNN: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyGRUModel, TestLoader, Device)
print(f'GRU: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyLSTMModel, TestLoader, Device)
print(f'LSTM: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')

RNN: MSE:447.7, L1:15.27, RMSE:21.16
GRU: MSE:206.27, L1:9.65, RMSE:14.36
LSTM: MSE:360.85, L1:13.55, RMSE:19.0


In [13]:
TestLoaderAttack = DataLoader(TestLoader.dataset, batch_size=100, shuffle=False)
X, y = next(iter(TestLoaderAttack))
X, y = X.to(Device), y.to(Device)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.06
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.02,
    }
}

print("RNN")
RNNAdversarialDataFgsm, RNNAdversarialDataBim, RNNAdversarialDataCW = AttacksTool.TestAttacks(MyRNNModel, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }
}

print("GRU")
GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.60
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 100,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0004,
    }
}

print("LSTM")
LSTMAdversarialDataFgsm, LSTMAdversarialDataBim, LSTMAdversarialDataCW = AttacksTool.TestAttacks(MyLSTMModel, X, y, AttacksParameters, Device = Device)

RNN
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 37.9299430847168, 'RMSE_adversarial': 0.05423940345644951, 'RMSE_pred': 35.62411880493164}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 11.520668029785156, 'RMSE_adversarial': 0.03503989800810814, 'RMSE_pred': 63.54457473754883}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 2.08231782913208, 'RMSE_adversarial': 0.034313660115003586, 'RMSE_pred': 74.24449920654297}

GRU
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 41.70381546020508, 'RMSE_adversarial': 0.011127319186925888, 'RMSE_pred': 29.44731903076172}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 40.320762634277344, 'RMSE_adversarial': 0.010623765178024769, 'RMSE_pred': 31.117568969726562}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 22.986116409301758, 'RMSE_adversarial': 0.011925701051950455, 'RMSE_pred': 53.595619201660156}

LSTM
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'P

In [32]:
AttackParameters = {
        "Epsilon": 0.02
    }

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_AdvTrainingFgsm = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "Fgsm", AttackParameters = AttackParameters,  Epochs = 80, Device="cuda", Verbose=0)

AttackParameters = {
        "Iterations": 30,
        "Epsilon": 0.001
    }
CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_AdvTrainingBim = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "Bim", AttackParameters = AttackParameters,  Epochs = 80, Device="cuda", Verbose=0)

AttackParameters = {
        "Iterations": 50,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_AdvTrainingCw = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "CW", AttackParameters = AttackParameters,  Epochs = 80 , Device="cuda", Verbose=0)

100%|██████████| 80/80 [16:37<00:00, 12.47s/it]


In [15]:
CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_EnsembleInputSqueezing = DefenseTool.EnsembleInputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0, MaxCoefficient=2)

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_InputSqueezing = DefenseTool.InputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0)

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_InputRandomization = DefenseTool.InputRandomization(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0)

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_InputPurificationAutoEncoder = DefenseTool.InputPurificationAutoEncoder(CurrentModel, TrainLoader, ValidationLoader, TestLoader, EpochsAutoEncoder = 30, EpochsModel = 30, Device="cuda", Verbose=0)

100%|██████████| 30/30 [02:05<00:00,  4.17s/it]


In [16]:
CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_EnsembleInputSqueezing = DefenseTool.EnsembleInputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0, MaxCoefficient=2)

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
Model_InputSqueezing = DefenseTool.InputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Coefficient = 0.1, Device="cuda",  Verbose=0)

100%|██████████| 30/30 [02:04<00:00,  4.17s/it]


In [17]:
AttackParameters = {
        "Epsilon": 0.01
    }

CurrentModel = GRUModel(NbFeatures, 12).to(Device)
DetectionModel = DefenseTool.GenerateDetectionModel(CurrentModel, TrainLoader, ValidationLoader, Device="cuda", Epochs = 30, LearningRate = 0.001,AttackParameters=AttackParameters, AdversarialMethod="Fgsm", Verbose=0)

100%|██████████| 30/30 [02:10<00:00,  4.35s/it]


In [18]:
Model_Distillation = DefenseTool.DefensiveDistillation(MyGRUModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=0)

100%|██████████| 30/30 [01:58<00:00,  3.96s/it]


In [19]:
RFModel, XGBModel = DefenseTool.EnsembleMethods(TrainLoader, ValidationLoader, TestLoader)

In [33]:
mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(MyGRUModel, TestLoader, Device)
print(f'Classicial : RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingFgsm, TestLoader, Device)
print(f'Adversarial training FGSM: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingBim, TestLoader, Device)
print(f'Adversarial training BIM: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingCw, TestLoader, Device)
print(f'Adversarial training CW: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_EnsembleInputSqueezing, TestLoader, Device)
print(f'Ensemble squeezing : RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputSqueezing, TestLoader, Device)
print(f'Squeezing: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputRandomization, TestLoader, Device)
print(f'Randomization: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputPurificationAutoEncoder, TestLoader, Device)
print(f'AutoEncoder: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_Distillation, TestLoader, Device)
print(f'Distillation: RMSE:{round(Rrmse,2)}')

DetectionRate = torch.round(DetectionModel(X.float())).sum()/len(X.float())
print("Detection rate: RMSE ", DetectionRate.item())

X_test, y_test = UtilsTool.DataLoaderToNumpy(TestLoader)
XTest = UtilsTool.Flatten_sequences(X.cpu().numpy())
MyRMSE = np.sqrt(mean_squared_error(RFModel.predict(XTest), y_test))
print(f'Random Forest test RMSE: {MyRMSE:.2f}')

X_test, y_test = UtilsTool.DataLoaderToNumpy(TestLoader)
XTest = UtilsTool.Flatten_sequences(X.cpu().numpy())
MyRMSE = np.sqrt(mean_squared_error(XGBModel.predict(XTest), y_test))
print(f'XGB RMSE: {MyRMSE:.2f}')

Classicial : RMSE:14.36
Adversarial training FGSM: RMSE:20.37
Adversarial training BIM: RMSE:22.51
Adversarial training CW: RMSE:22.22
Ensemble squeezing : RMSE:21.41
Squeezing: RMSE:17.92
Randomization: RMSE:14.53
AutoEncoder: RMSE:18.94
Distillation: RMSE:15.71
Detection rate: RMSE  0.0
Random Forest test RMSE: 14.16
XGB RMSE: 16.95


In [34]:
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW]
Models = [MyGRUModel, Model_AdvTrainingFgsm, Model_AdvTrainingBim, Model_AdvTrainingCw, Model_EnsembleInputSqueezing, Model_InputSqueezing, Model_InputRandomization, Model_InputPurificationAutoEncoder, Model_Distillation]
ModelsName = ["Classical", "Model_AdvTrainingFgsm", "Model_AdvTrainingBim", "Model_AdvTrainingCw", "Model_EnsembleInputSqueezing", "Model_InputSqueezing", "Model_InputRandomization", "Model_InputPurificationAutoEncoder", "Model_Distillation"]
EnsemblModels = [XGBModel, RFModel]
EnsemblModelsName = [ "XFG", "RF"]

UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)
UtilsTool.DisplayDetection(DetectionModel, AdversarialExamples)
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelsName, AdversarialExamples, y)

Classical
0 RMSE  14.36225414276123
1 RMSE  29.44731903076172
2 RMSE  31.117568969726562
3 RMSE  53.595619201660156

Model_AdvTrainingFgsm
0 RMSE  20.369470596313477
1 RMSE  16.026695251464844
2 RMSE  16.071840286254883
3 RMSE  24.9797306060791

Model_AdvTrainingBim
0 RMSE  22.51228141784668
1 RMSE  19.993263244628906
2 RMSE  19.9820556640625
3 RMSE  23.448793411254883

Model_AdvTrainingCw
0 RMSE  22.223268508911133
1 RMSE  22.12845230102539
2 RMSE  21.922340393066406
3 RMSE  20.174945831298828

Model_EnsembleInputSqueezing
0 RMSE  21.41400146484375
1 RMSE  21.694990158081055
2 RMSE  21.696256637573242
3 RMSE  29.076948165893555

Model_InputSqueezing
0 RMSE  17.922433853149414
1 RMSE  28.179719924926758
2 RMSE  27.568754196166992
3 RMSE  38.082637786865234

Model_InputRandomization
0 RMSE  17.019573211669922
1 RMSE  27.910348892211914
2 RMSE  26.116485595703125
3 RMSE  35.74281692504883

Model_InputPurificationAutoEncoder
0 RMSE  18.94183921813965
1 RMSE  18.371000289916992
2 RMSE  19.

In [22]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.010
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 105,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }
}
print("Classical Model")
_, _, _ = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model FGSM")
_, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingFgsm, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model BIM")
_, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingBim, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model CW")
_, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingCw, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model Ensemble squeezing")
_, _, _ = AttacksTool.TestAttacks(Model_EnsembleInputSqueezing, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model squeezing")
_, _, _ = AttacksTool.TestAttacks(Model_InputSqueezing, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000015,
    }
}

print("Adversarial Distillation")
_, _, _ = AttacksTool.TestAttacks(Model_Distillation, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.011
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 140,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000007,
    }
}
print("Adversarial Randomization")
_, _, _ = AttacksTool.TestAttacks(Model_InputRandomization, X, y, AttacksParameters, Device = Device)
print()
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    }
}
print("Adversarial Autoencoder")
_, _, _ = AttacksTool.TestAttacks(Model_InputPurificationAutoEncoder, X, y, AttacksParameters, Device = Device)

Classical Model
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 44.0766487121582, 'RMSE_adversarial': 0.0092728640884161, 'RMSE_pred': 27.123380661010742}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 42.32912826538086, 'RMSE_adversarial': 0.009346951730549335, 'RMSE_pred': 29.091541290283203}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 22.986116409301758, 'RMSE_adversarial': 0.011925701051950455, 'RMSE_pred': 53.595619201660156}

Adversarial Model FGSM
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 64.70822143554688, 'RMSE_adversarial': 0.008275505155324936, 'RMSE_pred': 16.089548110961914}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 63.73583984375, 'RMSE_adversarial': 0.00816867034882307, 'RMSE_pred': 16.298864364624023}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 24.760828018188477, 'RMSE_adversarial': 0.013250578194856644, 'RMSE_pred': 53.43511199951172}

Adversarial Model BIM
FGSM: {

In [23]:
SurrogateModelXGB = AttacksTool.CreateSurrogateModel(XGBModel, Epochs = 100, Device = "cuda", Verbose = 0)
SurrogateModelRF = AttacksTool.CreateSurrogateModel(RFModel, Epochs = 30, Device = "cuda", Verbose = 0)

100%|██████████| 30/30 [02:37<00:00,  5.26s/it]


In [24]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    }
}
A, B, C = AttacksTool.TestAttacks(SurrogateModelXGB, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
EnsemblModels = [XGBModel]
EnsemblModelNames = ["XGBModel"]
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelsName, AdversarialExamples, y)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    }
}
A, B, C = AttacksTool.TestAttacks(SurrogateModelRF, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
EnsemblModels = [RFModel]
EnsemblModelNames = ["RF"]
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelNames, AdversarialExamples, y)

FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 99.42424011230469, 'RMSE_adversarial': 0.011052585206925869, 'RMSE_pred': 41.838844299316406}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 99.5365219116211, 'RMSE_adversarial': 0.010664956644177437, 'RMSE_pred': 41.92062759399414}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 66.63797760009766, 'RMSE_adversarial': 0.011023596860468388, 'RMSE_pred': 31.324317932128906}
XFG
0 RMSE  16.952824
1 RMSE  21.592035
2 RMSE  21.464003
3 RMSE  43.892372
4 RMSE  18.847664
5 RMSE  18.313028
6 RMSE  48.79011

FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 87.1653823852539, 'RMSE_adversarial': 0.010904734022915363, 'RMSE_pred': 30.609468460083008}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 87.27487182617188, 'RMSE_adversarial': 0.010515548288822174, 'RMSE_pred': 30.70024299621582}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 33.564186096191406, 'RMSE_advers

In [35]:
#SurrogateSqueezing = AttacksTool.CreateSurrogateModelSqueezing(Model_InputSqueezing, Epochs = 300, LearningRate=0.01, Device ="cuda", Verbose = 0)
SurrogateSqueezing2 = AttacksTool.CreateSurrogateModelSqueezing(Model_EnsembleInputSqueezing, Epochs = 300, LearningRate=0.001, Device = "cuda", Verbose = 0)

 88%|████████▊ | 265/300 [1:01:55<08:02, 13.79s/it]

In [ ]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0003,
    }
}

A, B, C = AttacksTool.TestAttacks(SurrogateSqueezing, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
Models = [Model_EnsembleInputSqueezing]
ModelsName = ["Model_EnsembleInputSqueezing"]
UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)


AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00002,
    }
}
A, B, C = AttacksTool.TestAttacks(SurrogateSqueezing2, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
Models = [Model_InputSqueezing]
ModelsName = ["Model_InputSqueezing"]
UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)

FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 91.16725158691406, 'RMSE_adversarial': 0.010865531861782074, 'RMSE_pred': 39.140316009521484}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 88.52751159667969, 'RMSE_adversarial': 0.011064878664910793, 'RMSE_pred': 37.43061447143555}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.4576401114463806, 'RMSE_adversarial': 0.05241324380040169, 'RMSE_pred': 75.5243911743164}
Model_EnsembleInputSqueezing
0 RMSE  21.41400146484375
1 RMSE  21.694990158081055
2 RMSE  21.696256637573242
3 RMSE  29.076948165893555
4 RMSE  21.515195846557617
5 RMSE  21.366626739501953
6 RMSE  61.627342224121094

FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 29.176677703857422, 'RMSE_adversarial': 0.01084749773144722, 'RMSE_pred': 50.749698638916016}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 29.205724716186523, 'RMSE_adversarial': 0.01028687134385109, 'RMSE_pred': 50.717586517333984}
CW: 

In [27]:
Adv = AttacksTool.CwAttackDetection(MyGRUModel, DetectionModel, X, y, LearningRate= 0.00002, c = 1, d = 10000, Device = "cuda", Verbose =0)
mse = torch.mean((Adv - X) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")

mse = torch.mean((MyGRUModel(Adv) - y) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")
UtilsTool.DisplayDetection(DetectionModel, [Adv])

  0%|          | 0/1000 [00:00<?, ?it/s]/home/pierre/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([77])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 1000/1000 [00:01<00:00, 613.41it/s]

RMSE: 0.012999294977453951
RMSE: 33.495845794677734
0 Detection rate 0.0

